<a href="https://colab.research.google.com/github/willyptrain/TextAttackResearch/blob/main/%5BTextAttack%5D_Model_Example_AllenNLP_adapted_to_attack_api_branch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TextAttack & AllenNLP 

This is an example of testing adversarial attacks from TextAttack on pretrained models provided by AllenNLP. 

In a few lines of code, we load a sentiment analysis model trained on the Stanford Sentiment Treebank and configure it with a TextAttack model wrapper. Then, we initialize the TextBugger attack and run the attack on a few samples from the SST-2 train set.

For more information on AllenNLP pre-trained models: https://docs.allennlp.org/v1.0.0rc3/tutorials/getting_started/using_pretrained_models/

For more information about the TextBugger attack: https://arxiv.org/abs/1812.05271

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_2_allennlp.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_2_allennlp.ipynb)

In [1]:
!git clone -b attack-api https://github.com/QData/TextAttack.git
!pip install pyarrow --upgrade


Cloning into 'TextAttack'...
remote: Enumerating objects: 17776, done.
remote: Total 17776 (delta 0), reused 0 (delta 0), pack-reused 17776
Receiving objects: 100% (17776/17776), 23.67 MiB | 17.13 MiB/s, done.
Resolving deltas: 100% (13339/13339), done.
     |████████████████████████████████| 17.7MB 364kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [5]:
%cd TextAttack/
# !make
!python setup.py install
%cd ..

/content/TextAttack
running install
running bdist_egg
running egg_info
writing textattack.egg-info/PKG-INFO
writing dependency_links to textattack.egg-info/dependency_links.txt
writing entry points to textattack.egg-info/entry_points.txt
writing requirements to textattack.egg-info/requires.txt
writing top-level names to textattack.egg-info/top_level.txt
writing manifest file 'textattack.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/textattack
copying build/lib/textattack/attack_args.py -> build/bdist.linux-x86_64/egg/textattack
copying build/lib/textattack/attack.py -> build/bdist.linux-x86_64/egg/textattack
creating build/bdist.linux-x86_64/egg/textattack/augmentation
copying build/lib/textattack/augmentation/augmenter.py -> build/bdist.linux-x86_64/egg/textattack/augmentation
copying build/lib/textattack/augmentation/__init__.py -> build/bdi

In [2]:
!pip install allennlp allennlp_models 

     |████████████████████████████████| 512kB 12.5MB/s 
     |████████████████████████████████| 378kB 26.8MB/s 
     |████████████████████████████████| 266kB 13.5MB/s 
     |████████████████████████████████| 317kB 30.3MB/s 
     |████████████████████████████████| 1.4MB 37.8MB/s 
     |████████████████████████████████| 133kB 52.1MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 7.2MB 50.4MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.17.0-cp36-cp36m-linux_x86_64.whl size=3387898 sha256=11730ad8b3031a318032886361300982078524fb3170a3707704b915ea7c2e64
  Stored in directory: /root/.cache/pip/wheels/26/7a/37/7dbcc30a6b4efd17b91ad1f0128b7bbf84813bd4e1cfb8c1e3
  Created wheel for overrides: filename=overrides-3.1.0-cp36-none-any.whl size=10175 sha256=764b86f4ff5d7c9c1af4225884b94804e0240b7c22116f1dfa54eeee72f09849
  Stored in directory: /root/.cache/pip/wheels/5c/24/13/6ef8600e6f147

In [3]:
!pip install datasets transformers --upgrade

Requirement already up-to-date: datasets in /usr/local/lib/python3.6/dist-packages/datasets-1.2.0-py3.6.egg (1.2.0)
     |████████████████████████████████| 1.5MB 13.6MB/s 
ERROR: allennlp 1.3.0 has requirement transformers<4.1,>=4.0, but you'll have transformers 4.1.1 which is incompatible.
ERROR: textattack 0.2.15 has requirement numpy<1.19.0, but you'll have numpy 1.19.5 which is incompatible.
  Found existing installation: transformers 4.0.1
    Uninstalling transformers-4.0.1:
      Successfully uninstalled transformers-4.0.1


In [4]:
from allennlp.predictors import Predictor
import allennlp_models.classification

import textattack

class AllenNLPModel(textattack.models.wrappers.ModelWrapper):
    def __init__(self):
        self.model = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/basic_stanford_sentiment_treebank-2020.06.09.tar.gz")

    def __call__(self, text_input_list):
        outputs = []
        for text_input in text_input_list:
            outputs.append(self.model.predict(sentence=text_input))
        # For each output, outputs['logits'] contains the logits where
        # index 0 corresponds to the positive and index 1 corresponds 
        # to the negative score. We reverse the outputs (by reverse slicing,
        # [::-1]) so that negative comes first and positive comes second.
        return [output['logits'][::-1] for output in outputs]

model_wrapper = AllenNLPModel()

Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'


In [10]:
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextBuggerLi2018
from textattack.attacker import Attacker



dataset = HuggingFaceDataset("glue", "sst2", "train")
attack = TextBuggerLi2018.build(model_wrapper)

attacker = Attacker(attack, dataset)

results_iterable = attacker.attack_dataset()



# results = list(attack.attack_dataset(dataset, indices=range(20)))
# for idx, result in enumerate(results):
#     print(f'Result {idx}:')
#     print(result.__str__(color_method='ansi'))
#     print('\n')
# print()

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
textattack: Loading datasets dataset glue, subset sst2, split train.
Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7f586ddcc908>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



5 out of the last 17 calls to <function recreate_function.<locals>.restored_function_body at 0x7f5861596b70> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
[Succeeded / Failed / Total] 1 / 1 / 2:  40%|████      | 2/5 [00:00<00:00,  4.06it/s]

--------------------------------------------- Result 1 ---------------------------------------------
Negative (95%) --> Positive (93%)

hide new secretions from the parental units 

concealing new secretions from the parental units 


--------------------------------------------- Result 2 ---------------------------------------------
Negative (96%) --> [FAILED]

contains no wit , only labored gags 




[Succeeded / Failed / Total] 1 / 2 / 4:  80%|████████  | 4/5 [00:00<00:00,  4.44it/s]

--------------------------------------------- Result 3 ---------------------------------------------
Positive (100%) --> [FAILED]

that loves its characters and communicates something rather beautiful about human nature 


--------------------------------------------- Result 4 ---------------------------------------------
Positive (82%) --> [SKIPPED]

remains utterly satisfied to remain the same throughout 




[Succeeded / Failed / Total] 2 / 2 / 5: 100%|██████████| 5/5 [00:01<00:00,  3.31it/s]

--------------------------------------------- Result 5 ---------------------------------------------
Negative (98%) --> Positive (52%)

on the worst revenge-of-the-nerds clichés the filmmakers could dredge up 

on the pire reveng-of-the-nerds clichés the filmmakers could dragging up 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 40.0%  |
| Attack success rate:          | 50.0%  |
| Average perturbed word %:     | 22.14% |
| Average num. words per input: | 8.6    |
| Avg num queries:              | 25.5   |
+-------------------------------+--------+


In [2]:
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextBuggerLi2018

dataset = HuggingFaceDataset("glue", "sst2", "train")
attack = TextBuggerLi2018(model_wrapper)

results = list(attack.attack_dataset(dataset, indices=range(20)))
for idx, result in enumerate(results):
    print(f'Result {idx}:')
    print(result.__str__(color_method='ansi'))
    print('\n')
print()

textattack: Loading datasets dataset glue, subset sst2, split train.


Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4. Subsequent calls will reuse this data.


NameError: ignored